In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

c:\Users\jaesc2\GitHub\skforecast


In [2]:
import re
import pytest
import joblib
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.exceptions import NotFittedError
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import HistGradientBoostingRegressor
from lightgbm import LGBMRegressor

from skforecast.exceptions import IgnoredArgumentWarning
from skforecast.preprocessing import RollingFeatures
from skforecast.preprocessing import TimeSeriesDifferentiator
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries

from skforecast.ForecasterAutoreg.tests.fixtures_ForecasterAutoreg import data  # to test results when using differentiation


In [49]:
from skforecast.ForecasterAutoregMultiSeries.tests.fixtures_ForecasterAutoregMultiSeries import series
from skforecast.ForecasterAutoregMultiSeries.tests.fixtures_ForecasterAutoregMultiSeries import exog
from skforecast.ForecasterAutoregMultiSeries.tests.fixtures_ForecasterAutoregMultiSeries import exog_predict

series_dict = joblib.load(r'C:\Users\jaesc2\GitHub\skforecast\skforecast\ForecasterAutoregMultiSeries\tests\fixture_sample_multi_series.joblib')
exog_dict = joblib.load(r'C:\Users\jaesc2\GitHub\skforecast\skforecast\ForecasterAutoregMultiSeries\tests\fixture_sample_multi_series_exog.joblib')
end_train = "2016-07-31 23:59:00"
series_dict_train = {k: v.loc[:end_train,] for k, v in series_dict.items()}
exog_dict_train = {k: v.loc[:end_train,] for k, v in exog_dict.items()}
series_dict_test = {k: v.loc[end_train:,] for k, v in series_dict.items()}
exog_dict_test = {k: v.loc[end_train:,] for k, v in exog_dict.items()}

series_2 = pd.DataFrame({'1': pd.Series(np.arange(start=0, stop=50, dtype=float)), 
                         '2': pd.Series(np.arange(start=50, stop=100, dtype=float))})

In [62]:
forecaster = ForecasterAutoregMultiSeries(
    regressor          = LGBMRegressor(
        n_estimators=2, random_state=123, verbose=-1, max_depth=2
    ),
    lags               = 5,
    encoding           = None,
    dropna_from_series = False,
    transformer_series = StandardScaler(),
    transformer_exog   = StandardScaler()
)
forecaster.fit(
    series=series_dict_train, exog=exog_dict_train, suppress_warnings=True
)
levels = ['id_1000', 'id_1001', 'id_1003', 'id_1004', 'id_1005']
last_window = pd.DataFrame(
    {k: v for k, v in forecaster.last_window_.items() if k in levels}
)
last_window['id_1005'] = last_window['id_1004']
exog_dict_test_2 = exog_dict_test.copy()
exog_dict_test_2['id_1005'] = exog_dict_test_2['id_1004']
results = forecaster.create_predict_X(
    steps=5, levels=levels, last_window=last_window, exog=exog_dict_test_2, suppress_warnings=True
)
results['id_1000'].to_numpy()

array([[-0.98327068, -0.88248605, -0.79842631, -0.79277912, -0.77768307,
         0.00821644,  1.42962482,         nan,         nan],
       [-0.12514786, -0.98327068, -0.88248605, -0.79842631, -0.79277912,
         1.11220226,  0.89634375,         nan,         nan],
       [-0.12514786, -0.12514786, -0.98327068, -0.88248605, -0.79842631,
         1.38486425, -0.30192795,         nan,         nan],
       [-0.07645311, -0.12514786, -0.12514786, -0.98327068, -0.88248605,
         0.62088235, -1.26286725,         nan,         nan],
       [-0.07645311, -0.07645311, -0.12514786, -0.12514786, -0.98327068,
        -0.60444947, -1.26286725,         nan,         nan]])

In [67]:
results['id_1005'].to_numpy()

array([[ 1.22899343,  1.65819658,  2.4318926 ,  2.56743042,  2.38367241,
         0.00821644,  1.42962482,  1.11141113, -0.87943526],
       [ 0.12663112,  1.22899343,  1.65819658,  2.4318926 ,  2.56743042,
         1.11220226,  0.89634375,  1.1327558 ,  0.0058948 ],
       [ 0.01290134,  0.12663112,  1.22899343,  1.65819658,  2.4318926 ,
         1.38486425, -0.30192795,  1.1775869 , -0.3532584 ],
       [ 0.01290134,  0.01290134,  0.12663112,  1.22899343,  1.65819658,
         0.62088235, -1.26286725,  1.0428337 ,  0.84287284],
       [ 0.01290134,  0.01290134,  0.01290134,  0.12663112,  1.22899343,
        -0.60444947, -1.26286725,  1.00599776, -0.62314633]])

In [4]:
end_train = '2003-03-01 23:59:00'

# Simulated exogenous variable
rng = np.random.default_rng(9876)
exog = pd.Series(
    rng.normal(loc=0, scale=1, size=len(data)), index=data.index, name='exog'
)
rolling = RollingFeatures(stats=['mean', 'median'], window_sizes=[5, 5])
rolling_2 = RollingFeatures(stats='sum', window_sizes=[6])

forecaster = ForecasterAutoregDirect(
                    regressor        = LinearRegression(),
                    lags             = [1, 5],
                    window_features  = [rolling, rolling_2],
                    steps            = 6,
                    transformer_y    = None,
                    transformer_exog = None,
                    differentiation  = None
                )
forecaster.fit(y=data.loc[:end_train], exog=exog.loc[:end_train])
X_predict = forecaster.create_predict_X(exog=exog.loc[end_train:])
X_predict

,lag_1,lag_5,roll_mean_5,roll_median_5,roll_sum_6,exog
2003-04-01,2.043333,2.366796,2.261869,2.366796,13.686322,1.161729
2003-05-01,2.043333,2.366796,2.261869,2.366796,13.686322,0.294688
2003-06-01,2.043333,2.366796,2.261869,2.366796,13.686322,-0.439976
2003-07-01,2.043333,2.366796,2.261869,2.366796,13.686322,1.250084
2003-08-01,2.043333,2.366796,2.261869,2.366796,13.686322,1.374969
2003-09-01,2.043333,2.366796,2.261869,2.366796,13.686322,-0.416732


In [8]:
for i, step in enumerate(range(1, forecaster.steps + 1)):
    print(forecaster.regressors_[step].predict(X_predict.iloc[[i]]))

[2.19238587]
[2.23555505]
[2.25654547]
[2.34763646]
[2.31040474]
[2.43900132]


In [10]:
forecaster.predict(steps=[2], exog=exog.loc[end_train:]).to_numpy()

array([2.23555505])

In [11]:
import pytest
import re
import numpy as np
import pandas as pd
from skforecast.model_selection._split import BaseFold

In [12]:
valid_params = {
    "steps": 5,
    "initial_train_size": 100,
    "window_size": 10,
    "differentiation": None,
    "refit": True,
    "fixed_train_size": True,
    "gap": 0,
    "skip_folds": None,
    "allow_incomplete_fold": True,
    "return_all_indexes": False,
    "verbose": True,
}

In [13]:
cv = BaseFold()
params = dict(valid_params)
params["initial_train_size"] = "invalid"
msg = (
    f"`initial_train_size` must be an integer greater than 0 or None. "
    f"Got {params["initial_train_size"]}."
)
with pytest.raises(ValueError, match=msg):
    cv._validate_params(cv_name="TimeSeriesFold", **params)

In [15]:
params is valid_params

False

In [14]:
params

{'steps': 5,
 'initial_train_size': 'invalid',
 'window_size': 10,
 'differentiation': None,
 'refit': True,
 'fixed_train_size': True,
 'gap': 0,
 'skip_folds': None,
 'allow_incomplete_fold': True,
 'return_all_indexes': False,
 'verbose': True}